Once the simulations are completed, the results files are read and the important information is extracted and loaded into a .csv file. 

In [1]:
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt

Test_Cases = pd.read_csv("Test_Cases.csv", sep=',')

In [2]:

def diagnostic(log):
    log = open(log,'r')
    data = log.readlines()
    data = data[17:]

    # Sol steps
    lookup = " Step"
    steps = []
    for num, line in enumerate(data):
        if lookup in line:
            steps.append(num)
    fin = steps[-1]

    if len(steps) > 1:
        for i in range(len(steps)-1):
            steps[i] = steps[i+1] - steps[i]
        steps.pop()
        steps = np.array(steps) - np.full(len(steps), 1)
    else:
        steps = [len(data) - 3]

    
    #Initialize contents of interest
    Time = 0
    Memory = 0
    Avg_Iterations_perTS = np.mean(steps)
    Completed = False
    Still_Running = False
    Plastic_Conv_Err = False
    Conv_Iter_Err = False
    Conv_Error = []

    #Time/memory of run
    t = re.findall(r'\d+', data[-2])
    m = re.findall(r'\d+', data[-1])
    if len(t) == 2:
        Time = int(t[0])*60 + int(t[1])
        Memory = int(m[0])
    elif len(t) == 3:
        Time = int(t[0])*60*60 + int(t[1])*60 + int(t[2])
        Memory = int(m[0])
    else:
        Time = int(t[0])
        Memory = int(m[0])

    
    #Check for solution completion
    end = re.findall(r'\d+', data[fin])
    if end[0] == end[1]:
        Completed = True
    else:
        if data[fin] == data[-1]:
            Still_Running = True
        else:
            #Plasticity convergence error
            p = data[fin+1]
            if p == "\n":
                Plastic_Conv_Err = True

            # Non-convergence stats
            if Completed == 0 & Plastic_Conv_Err == 0:
                Conv_Iter_Err = True
                for line in data:
                    if "Warning:" in line:
                        Conv_Error = re.findall("\d+\.\d+", line)
                        Conv_Error = float(Conv_Error[0])
            
        
    return [Time, Memory, Avg_Iterations_perTS, Completed, Still_Running, Plastic_Conv_Err, Conv_Iter_Err, Conv_Error]



count = 0
num_strain = 14
num_timesteps = 5
log_diagnostic = []
for i in range(num_strain):
    for j in range(num_timesteps):
        l = "Test_RVEv_" + str(i) + str(j) +".log"
        log_diagnostic.append(diagnostic(l))
        count = count + 1


In [3]:
def failure_analysis(csv):
    df = pd.read_csv(csv)

    #stress strain testing if component failed
    S11 = df[' Stress_11'].iloc[-1] != max(df[' Stress_11'])
    S22 = df[' Stress_22'].iloc[-1] != max(df[' Stress_22'])
    S33 = df[' Stress_33'].iloc[-1] != max(df[' Stress_33'])
    S12 = df[' Stress_12'].iloc[-1] != max(df[' Stress_12'])
    S13 = df[' Stress_13'].iloc[-1] != max(df[' Stress_13'])
    S23 = df[' Stress_23'].iloc[-1] != max(df[' Stress_23'])
    
    #Evaluating how "Failed" the component is
    F11 = df[' Stress_11'].iloc[-1] / max(df[' Stress_11'])
    F22 = df[' Stress_22'].iloc[-1] / max(df[' Stress_22'])
    F33 = df[' Stress_33'].iloc[-1] / max(df[' Stress_33'])
    F12 = df[' Stress_12'].iloc[-1] / max(df[' Stress_12'])
    F13 = df[' Stress_13'].iloc[-1] / max(df[' Stress_13'])
    F23 = df[' Stress_23'].iloc[-1] / max(df[' Stress_23'])
    
    #End strain of the simulation
    E11 = df[' Strain_11'].iloc[-1]
    E22 = df[' Strain_22'].iloc[-1]
    E33 = df[' Strain_33'].iloc[-1]
    E12 = df[' Strain_12'].iloc[-1]
    E13 = df[' Strain_13'].iloc[-1]
    E23 = df[' Strain_23'].iloc[-1]
    
    return [S11, S22, S33, S12, S13, S23, F11, F22, F33, F12, F13, F23, E11, E22, E33, E12, E13, E23]


count = 0
num_strain = 14
num_timesteps = 5
failure = []
for i in range(num_strain):
    for j in range(num_timesteps):
        c = "Test_RVEv_" + str(i) + str(j) +".csv"
        failure.append(failure_analysis(c))
        count = count + 1


In [4]:
#Build appropriate csv files for the analysis
log_df = pd.DataFrame(log_diagnostic, 
                   columns = ["Time", "Memory", "Avg_Iterations_perTS", \
                              "Completed", "Still Running", "Plastic Conv. Err", "Conv. Iter. Err", "Conv. Err"])
log_df.to_csv("Log_Diagnostic.csv")
          
    
failure_df = pd.DataFrame(failure, 
                   columns = ["Failed S11", "Failed S22", "Failed S33", "Failed S12", "Failed S13", "Failed S23", \
                             "F11", "F22", "F33", "F12", "F13", "F23", "E11", "E22", "E33", "E12", "E13", "E23"])
failure_df.to_csv("Failure_Diagnostic.csv")